# Weather Forecast Data Download (Nov 20-29, 2025)

This notebook downloads hourly weather **forecast** data from Open-Meteo API for multiple load areas across the PJM region.
Data is retrieved in UTC and converted to Eastern Time (ET).

**Date Range**: November 20, 2025 00:00 UTC to November 29, 2025 23:00 UTC  
**Note**: Uses forecast API for future dates

## 1. Import Required Libraries

In [3]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import os
import time
import holidays
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import warnings
import holidays
import pickle

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Libraries imported successfully")

Libraries imported successfully


## 2. Define Load Area Coordinates

Define the latitude and longitude for each load area in the PJM region.

In [5]:
zone_coords = pd.DataFrame({
    'load_area': ['AECO', 'AEPAPT', 'AEPIMP', 'AEPKPT', 'AEPOPT', 'AP', 'BC', 'CE', 'DAY', 'DEOK',
                  'DOM', 'DPLCO', 'DUQ', 'EASTON', 'EKPC', 'JC', 'ME', 'OE', 'OVEC', 'PAPWR',
                  'PE', 'PEPCO', 'PLCO', 'PN', 'PS', 'RECO', 'SMECO', 'UGI', 'VMEU'],
    'lat': [39.45, 37.25, 38.45, 38.20, 39.90, 37.30, 40.80, 41.85, 39.75, 39.10,
            37.55, 38.90, 40.45, 39.55, 37.75, 40.35, 40.20, 41.10, 38.85, 40.70,
            40.00, 38.90, 40.95, 41.15, 40.75, 41.00, 38.40, 40.25, 37.30],
    'lon': [-74.50, -81.30, -81.60, -83.10, -82.90, -80.90, -79.95, -86.10, -84.20, -84.50,
            -77.45, -75.50, -79.90, -75.10, -84.30, -74.65, -76.00, -81.25, -82.85, -77.80,
            -75.20, -76.95, -77.40, -77.80, -74.15, -74.10, -76.70, -75.65, -76.00]
})

print(f"Total load areas: {len(zone_coords)}")
zone_coords.head()

Total load areas: 29


,load_area,lat,lon
0,AECO,39.45,-74.5
1,AEPAPT,37.25,-81.3
2,AEPIMP,38.45,-81.6
3,AEPKPT,38.20,-83.1
4,AEPOPT,39.90,-82.9


## 3. Configure API Parameters

In [7]:
# Date range to download
start_date = "2025-11-20"
end_date = "2025-11-30"

# API endpoint - using forecast API for future dates
api = "https://api.open-meteo.com/v1/forecast"

# Weather variables to retrieve
hourly_vars = "temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,wind_speed_10m"

print(f"Date range: {start_date} to {end_date}")
print(f"Variables: {hourly_vars}")
print(f"Using forecast API for future dates")

Date range: 2025-11-20 to 2025-11-30
Variables: temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,wind_speed_10m
Using forecast API for future dates


## 4. Define Weather Data Fetcher Function

In [9]:
def fetch_weather_data(lat, lon, start_date, end_date, tries=3):
    """Fetch weather data for a specific date range and location"""
    # For forecast API, we use forecast_days parameter (max 16 days)
    url = (f"{api}?latitude={lat}&longitude={lon}"
           f"&hourly={hourly_vars}&timezone=UTC&forecast_days=16")
    
    for k in range(1, tries + 1):
        try:
            res = requests.get(url, timeout=60)
            if res.status_code == 200:
                j = res.json()
                if 'hourly' in j and 'time' in j['hourly']:
                    df = pd.DataFrame({
                        'datetime_beginning_utc': j['hourly']['time'],
                        'temp': j['hourly']['temperature_2m'],
                        'humidity': j['hourly']['relative_humidity_2m'],
                        'dew_point': j['hourly']['dew_point_2m'],
                        'precip': j['hourly']['precipitation'],
                        'wind': j['hourly']['wind_speed_10m']
                    })
                    
                    # Filter to the specific date range we want
                    df['datetime_beginning_utc'] = pd.to_datetime(df['datetime_beginning_utc'])
                    mask = (df['datetime_beginning_utc'] >= start_date) & (df['datetime_beginning_utc'] <= end_date + ' 23:00:00')
                    df = df[mask].copy()
                    df['datetime_beginning_utc'] = df['datetime_beginning_utc'].astype(str)
                    
                    return df
                else:
                    print(f"Unexpected response format for lat={lat}, lon={lon}")
            else:
                print(f"HTTP {res.status_code} for lat={lat}, lon={lon}")
        except Exception as e:
            print(f"Error on attempt {k} for lat={lat}, lon={lon}: {e}")
        
        if k < tries:
            time.sleep(0.7 * k)
    
    return None

## 5. Download Weather Data for All Load Areas

This cell downloads weather data for the specified date range across all load areas.

In [11]:
# Initialize list to collect data from all zones
all_data = []

print(f"Downloading weather data for {len(zone_coords)} load areas...\n")
print("="*60)

for idx, row in zone_coords.iterrows():
    zone = row['load_area']
    lat = row['lat']
    lon = row['lon']
    
    print(f"[{idx+1}/{len(zone_coords)}] Fetching {zone} (lat={lat}, lon={lon})...")
    
    # Fetch data for this zone
    df = fetch_weather_data(lat, lon, start_date, end_date)
    
    if df is not None and not df.empty:
        # Add load area identifier
        df['load_area'] = zone
        all_data.append(df)
        print(f"  ✓ Successfully fetched {len(df)} hourly records")
    else:
        print(f"  ✗ Failed to fetch data for {zone}")
    
    # Small delay between requests to be respectful to the API
    time.sleep(0.5)

print("="*60)
print(f"\nData collection complete! Total zones processed: {len(all_data)}/{len(zone_coords)}")


[1/29] Fetching AECO (lat=39.45, lon=-74.5)...
  ✓ Successfully fetched 240 hourly records
[2/29] Fetching AEPAPT (lat=37.25, lon=-81.3)...
  ✓ Successfully fetched 240 hourly records
[3/29] Fetching AEPIMP (lat=38.45, lon=-81.6)...
  ✓ Successfully fetched 240 hourly records
[4/29] Fetching AEPKPT (lat=38.2, lon=-83.1)...
  ✓ Successfully fetched 240 hourly records
[5/29] Fetching AEPOPT (lat=39.9, lon=-82.9)...
  ✓ Successfully fetched 240 hourly records
[6/29] Fetching AP (lat=37.3, lon=-80.9)...
  ✓ Successfully fetched 240 hourly records
[7/29] Fetching BC (lat=40.8, lon=-79.95)...
  ✓ Successfully fetched 240 hourly records
[8/29] Fetching CE (lat=41.85, lon=-86.1)...
  ✓ Successfully fetched 240 hourly records
[9/29] Fetching DAY (lat=39.75, lon=-84.2)...
  ✓ Successfully fetched 240 hourly records
[10/29] Fetching DEOK (lat=39.1, lon=-84.5)...
  ✓ Successfully fetched 240 hourly records
[11/29] Fetching DOM (lat=37.55, lon=-77.45)...
  ✓ Successfully fetched 240 hourly records

## 6. Combine Data and Convert to Eastern Time

Combine all load area data and convert UTC timestamps to Eastern Time.

In [13]:
if all_data:
    # Combine all data
    combined_df = pd.concat(all_data, ignore_index=True)
    
    print(f"Combined dataset shape: {combined_df.shape}")
    print(f"Date range (UTC): {combined_df['datetime_beginning_utc'].min()} to {combined_df['datetime_beginning_utc'].max()}")
    
    # Convert UTC to Eastern Time
    print("\nConverting UTC to Eastern Time...")
    combined_df['datetime_beginning_utc'] = pd.to_datetime(combined_df['datetime_beginning_utc'])
    
    # Create Eastern Time column
    utc = pytz.UTC
    eastern = pytz.timezone('US/Eastern')
    et_time = combined_df['datetime_beginning_utc'].dt.tz_localize(utc).dt.tz_convert(eastern)
    
    # Format datetime in M/D/YYYY H:MM:SS AM/PM format
    # Remove timezone info and format manually for cross-platform compatibility
    et_time_no_tz = et_time.dt.tz_localize(None)
    
    # Try Unix format first, fall back to Windows format if needed
    try:
        combined_df['datetime_beginning_ept'] = et_time_no_tz.dt.strftime('%-m/%-d/%Y %-I:%M:%S %p')
    except:
        # Windows format
        combined_df['datetime_beginning_ept'] = et_time_no_tz.dt.strftime('%#m/%#d/%Y %#I:%M:%S %p')
    
    # Remove UTC column and reorder
    combined_df = combined_df.drop('datetime_beginning_utc', axis=1)
    column_order = ['datetime_beginning_ept', 'load_area', 'temp', 'humidity', 'dew_point', 'precip', 'wind']
    combined_df = combined_df[column_order]
    
    print("✓ Time conversion complete")
    
    # Display sample
    print("\nSample of data:")
    display(combined_df.head(10))
else:
    print("No data was successfully downloaded!")

Combined dataset shape: (6960, 7)
Date range (UTC): 2025-11-21 00:00:00 to 2025-11-30 23:00:00

Converting UTC to Eastern Time...
✓ Time conversion complete

Sample of data:


,datetime_beginning_ept,load_area,temp,humidity,dew_point,precip,wind
0,11/20/2025 7:00:00 PM,AECO,6.4,72,1.7,0.0,8.1
1,11/20/2025 8:00:00 PM,AECO,6.2,77,2.5,0.0,6.3
2,11/20/2025 9:00:00 PM,AECO,6.8,79,3.4,0.0,3.8
3,11/20/2025 10:00:00 PM,AECO,7.0,81,4.0,0.0,4.2
4,11/20/2025 11:00:00 PM,AECO,7.3,76,3.3,0.0,3.1
5,11/21/2025 12:00:00 AM,AECO,7.2,76,3.3,0.0,0.5
6,11/21/2025 1:00:00 AM,AECO,7.3,83,4.6,0.0,3.7
7,11/21/2025 2:00:00 AM,AECO,7.2,86,5.0,0.0,3.5
8,11/21/2025 3:00:00 AM,AECO,7.4,82,4.6,0.0,2.4
9,11/21/2025 4:00:00 AM,AECO,7.6,81,4.6,0.0,0.8


In [14]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Directory paths (relative to src/)
FIGURES_DIR = "../figures"
OUTPUT_DIR = "../output"

# Create directories if they don't exist
os.makedirs(FIGURES_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Target 29 load areas
KEEP_AREAS = [
    "AECO", "AEPAPT", "AEPIMP", "AEPKPT", "AEPOPT", "AP", "BC", "CE", "DAY", "DEOK",
    "DOM", "DPLCO", "DUQ", "EASTON", "EKPC", "JC", "ME", "OE", "OVEC", "PAPWR",
    "PE", "PEPCO", "PLCO", "PN", "PS", "RECO", "SMECO", "UGI", "VMEU"
]

# Best Model Finding dates (2025 test)
TEST_START = '2025-11-20 00:00:00'
TEST_END = '2025-11-29 23:00:00'

# Rolling window for peak days
WINDOW_SIZE = 10  # days
NUM_PEAK_DAYS = 2

In [15]:
df= combined_df.copy()
def parse_et(series):
    """Parse datetime with Eastern Time timezone handling"""
    if pd.api.types.is_datetime64_any_dtype(series):
        result = pd.to_datetime(series)
        if result.dt.tz is None:
            return result.dt.tz_localize('America/New_York', ambiguous='NaT', nonexistent='NaT')
        return result
    result = pd.to_datetime(series, format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
    mask = result.isna()
    if mask.any():
        result[mask] = pd.to_datetime(series[mask], errors='coerce')
    return result.dt.tz_localize('America/New_York', ambiguous='NaT', nonexistent='NaT')

df = df.rename(columns={
    'datetime_beginning_ept': 'datetime',
    'load_area': 'region',
    'temp': 'temperature',
    'precip': 'precipitation',
    'wind': 'wind_speed'
})

# Keep only necessary columns
keep_cols = ['datetime', 'region', 'temperature', 'humidity', 'precipitation', 'wind_speed']
df = df[keep_cols]

df['datetime'] = parse_et(df['datetime'])
df = df.dropna(subset=['datetime'])
df = df.sort_values('datetime').reset_index(drop=True)

In [16]:
def add_features(df):
    """Add temporal and calendar features"""
    df = df.copy()
    
    # ===== TEMPORAL FEATURES =====
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek  # Monday=0, Sunday=6
    df['month'] = df['datetime'].dt.month
    df['day_of_month'] = df['datetime'].dt.day
    df['day_of_year'] = df['datetime'].dt.dayofyear
    df['week_of_year'] = df['datetime'].dt.isocalendar().week
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    # Cyclical encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    # ===== HOLIDAY FEATURES =====
    # Create US holiday calendar
    us_holidays = holidays.US(years=range(2016, 2026))
    df['date'] = df['datetime'].dt.date
    df['is_holiday'] = df['date'].apply(lambda x: int(x in us_holidays))
    
    # Day before/after holiday
    df['is_day_before_holiday'] = df['is_holiday'].shift(-24).fillna(0).astype(int)
    df['is_day_after_holiday'] = df['is_holiday'].shift(24).fillna(0).astype(int)
    
    # Thanksgiving - Cooking load, midday peak
    df['is_thanksgiving'] = df['date'].apply(
        lambda x: int(1 if us_holidays.get(x) == 'Thanksgiving' else 0)
    )
    
    # Christmas - Low commercial, high residential heating
    df['is_christmas'] = df['date'].apply(
        lambda x: int(1 if us_holidays.get(x) == 'Christmas Day' else 0)
    )
    
    # New Year's Day - Late night/early morning shift
    df['is_new_years'] = df['date'].apply(
        lambda x: int(1 if us_holidays.get(x) == "New Year's Day" else 0)
    )
    
    # July 4th - Summer, outdoor, evening grilling/fireworks
    df['is_july4'] = df['date'].apply(
        lambda x: int(1 if us_holidays.get(x) == 'Independence Day' else 0)
    )
    # Others
    df['is_other_holiday'] = (df['is_holiday']-df['is_thanksgiving']-df['is_christmas']-df['is_new_years']-df['is_july4'])
    
    return df

df = add_features(df)

# Task 1

In [18]:
FEATURE_COLS = [
    # Temporal
    'hour', 'day_of_week', 'month', 'day_of_month', 'day_of_year', 'week_of_year', 'is_weekend',
    'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
    # Weather
    'temperature', 'humidity', 'wind_speed', 'precipitation',
    # 'is_holiday', 'is_day_before_holiday', 'is_day_after_holiday'
    'is_thanksgiving', 'is_christmas', 'is_new_years', 'is_july4', 'is_other_holiday', 'is_day_before_holiday', 'is_day_after_holiday'
]

In [19]:
test_start = pd.to_datetime(TEST_START)
test_end = pd.to_datetime(TEST_END)

def prepare_data(df, test_start, test_end, FEATURE_COLS, region=None):
    """
    Prepare train and test data for a given time period and region.
    """
    # Filter by region if specified
    if region is not None:
        df = df[df['region'] == region].copy()
    
    # Convert string dates to datetime
    test_start_dt = pd.Timestamp(test_start).tz_localize('America/New_York')
    test_end_dt = pd.Timestamp(test_end).tz_localize('America/New_York')
    
    # Split data
    test_data = df[(df['datetime'] >= test_start_dt) & (df['datetime'] <= test_end_dt)].copy()
    
    # Drop rows with missing lag features
    test_data = test_data.dropna(subset=FEATURE_COLS)
    
    # Prepare X and y
    X_test = test_data[FEATURE_COLS].values
    
    return X_test, test_data

print("Helper functions defined")

Helper functions defined


In [20]:
# Load the saved models
regions= ['AECO', 'AEPAPT', 'AEPIMP', 'AEPKPT', 'AEPOPT', 'AP', 'BC', 'CE', 'DAY', 'DEOK',
                  'DOM', 'DPLCO', 'DUQ', 'EASTON', 'EKPC', 'JC', 'ME', 'OE', 'OVEC', 'PAPWR',
                  'PE', 'PEPCO', 'PLCO', 'PN', 'PS', 'RECO', 'SMECO', 'UGI', 'VMEU']
models_dir = os.path.join(OUTPUT_DIR, 'trained_models')
model_path = os.path.join(models_dir, 'hourly_load_models.pkl')

print("Loading saved models...")
with open(model_path, 'rb') as f:
    hourly_best_models = pickle.load(f)
print(f"Loaded {len(hourly_best_models)} regional models")

# Generate predictions for all regions
predictions_list = []

for region in tqdm(regions, desc="Generating predictions"):
    # Get model info for this region
    model_info = hourly_best_models[region]
    model = model_info['model']
    feature_cols = model_info['feature_cols']
    method = model_info['method']
    
    # Prepare prediction data
    pred_data = df[df['region'] == region].copy()
    
    if len(pred_data) == 0:
        print(f"Warning: No data for {region} in prediction period")
        continue
    
    # Get features
    X_pred = pred_data[feature_cols]
    
    y_pred = model.predict(X_pred)
    y_pred = np.round(y_pred).astype(int)
    # Store predictions with metadata
    pred_df = pred_data[['date']].copy()
    pred_df['region'] = region
    pred_df['predicted_load'] = y_pred
    pred_df['hour'] = pred_data['hour']
    predictions_list.append(pred_df)

# Combine all predictions
all_predictions = pd.concat(predictions_list, ignore_index=True)

print(f"\nTotal predictions generated: {len(all_predictions):,}")

# Create 3D structure: Pivot to get day x region x hour
predictions_pivot = all_predictions.pivot_table(
    index='date',
    columns=['region', 'hour'],
    values='predicted_load'
)

print(f"\nPredictions shape: {predictions_pivot.shape}")
print(f"  Days: {len(predictions_pivot.index)}")
print(f"  Regions: {len(regions)}")
print(f"  Hours: 24")

# Alternative: Create a proper 3D array
dates = sorted(all_predictions['date'].unique())
hours = list(range(24))

# Initialize 3D array: (days, regions, hours)
predictions_3d = np.zeros((len(dates), len(regions), len(hours)))

# Fill the 3D array
for i, date in enumerate(dates):
    for j, region in enumerate(regions):
        for k, hour in enumerate(hours):
            mask = (all_predictions['date'] == date) & \
                   (all_predictions['region'] == region) & \
                   (all_predictions['hour'] == hour)
            
            if mask.sum() > 0:
                predictions_3d[i, j, k] = all_predictions.loc[mask, 'predicted_load'].values[0]
            else:
                predictions_3d[i, j, k] = np.nan

print(f"\n3D Array shape: {predictions_3d.shape}")
print(f"  Dimension 0 (days): {predictions_3d.shape[0]}")
print(f"  Dimension 1 (regions): {predictions_3d.shape[1]}")
print(f"  Dimension 2 (hours): {predictions_3d.shape[2]}")

all_predictions['predicted_load'] = all_predictions['predicted_load'].astype(int)
# Create a more accessible DataFrame format
print("\nCreating multi-index DataFrame...")
predictions_df = all_predictions.pivot_table(
    index=['date', 'region'],
    columns='hour',
    values='predicted_load'
).reset_index()

# Rename hour columns for clarity
hour_cols = {i: f'hour_{i:02d}' for i in range(24)}
predictions_df = predictions_df.rename(columns=hour_cols)

print(f"Predictions DataFrame shape: {predictions_df.shape}")
print(f"Columns: {list(predictions_df.columns[:5])}...")

print("\n" + "="*70)
print("Prediction generation complete!")
print("="*70)

Loading saved models...
Loaded 29 regional models


Generating predictions: 100%|██████████████████| 29/29 [00:00<00:00, 610.87it/s]


Total predictions generated: 6,960

Predictions shape: (11, 696)
  Days: 11
  Regions: 29
  Hours: 24



3D Array shape: (11, 29, 24)
  Dimension 0 (days): 11
  Dimension 1 (regions): 29
  Dimension 2 (hours): 24

Creating multi-index DataFrame...
Predictions DataFrame shape: (319, 26)
Columns: ['date', 'region', 'hour_00', 'hour_01', 'hour_02']...

Prediction generation complete!


# Task 2

In [22]:
# Load the saved peak hour models
regions = ['AECO', 'AEPAPT', 'AEPIMP', 'AEPKPT', 'AEPOPT', 'AP', 'BC', 'CE', 'DAY', 'DEOK',
           'DOM', 'DPLCO', 'DUQ', 'EASTON', 'EKPC', 'JC', 'ME', 'OE', 'OVEC', 'PAPWR',
           'PE', 'PEPCO', 'PLCO', 'PN', 'PS', 'RECO', 'SMECO', 'UGI', 'VMEU']

models_dir = os.path.join(OUTPUT_DIR, 'trained_models')
model_path = os.path.join(models_dir, 'peak_hour_models.pkl')

print("Loading saved peak hour prediction models...")
with open(model_path, 'rb') as f:
    peak_hour_best_models = pickle.load(f)
print(f"Loaded {len(peak_hour_best_models)} regional models")


# Dictionary to store peak hour predictions
peak_hour_predictions = {}

for region in tqdm(regions, desc="Predicting peak hours"):
    # Get model info for this region
    model_info = peak_hour_best_models[region]
    model = model_info['model']
    feature_cols = model_info['feature_cols']
    method = model_info['method']
    
    # Prepare prediction data for this region
    pred_data = df[df['region'] == region].copy()
    
    if len(pred_data) == 0:
        print(f"Warning: No data for {region} in prediction period")
        continue
    
    # Get features
    X_pred = pred_data[feature_cols]
    
    # Get predicted probabilities for being peak hour
    y_pred_proba = model.predict_proba(X_pred)[:, 1]  # Probability of class 1 (is_peak_hour=True)
    
    # Add predictions to dataframe
    pred_data['peak_hour_probability'] = y_pred_proba
    
    # For each date, find the hour with maximum probability
    region_peak_hours = {}
    for date in pred_data['date'].unique():
        date_data = pred_data[pred_data['date'] == date]
        # Find hour with maximum probability
        peak_hour_idx = date_data['peak_hour_probability'].idxmax()
        peak_hour = date_data.loc[peak_hour_idx, 'hour']
        region_peak_hours[date] = int(peak_hour)
    
    peak_hour_predictions[region] = region_peak_hours

# Convert to 2D DataFrame (regions × dates)
print("\nCreating 2D DataFrame (regions × dates)...")

# Get all unique dates
all_dates = sorted(df['date'].unique())

# Create DataFrame with regions as rows and dates as columns
peak_hour_df = pd.DataFrame(peak_hour_predictions).T
peak_hour_df = peak_hour_df.sort_index()  # Sort by region name

# Ensure columns are sorted by date
peak_hour_df = peak_hour_df[sorted(peak_hour_df.columns)]

# Display the predictions
print("\nPeak Hour Predictions (hour of day, 0-23):")
print(peak_hour_df)

Loading saved peak hour prediction models...
Loaded 29 regional models


Predicting peak hours: 100%|███████████████████| 29/29 [00:00<00:00, 311.77it/s]


Creating 2D DataFrame (regions × dates)...

Peak Hour Predictions (hour of day, 0-23):
        2025-11-20  2025-11-21  2025-11-22  2025-11-23  2025-11-24  \
AECO            19          17          18          18          18   
AEPAPT          19          18          10           8           7   
AEPIMP          19          10          19          18           7   
AEPKPT          20           9           8           8           7   
AEPOPT          19          10          18          18          18   
AP              19          17          18          18           7   
BC              19          17          18          18           7   
CE              19          18          18          18          18   
DAY             19           9          18          18          18   
DEOK            19          17          18          18          18   
DOM             19           7          17          18           7   
DPLCO           19          17          17          18           7   
DU

# Task 3

In [24]:
feature_cols = [
    # Temperature features
    'temperature_max', 'temperature_min', 'temperature_mean', 'temperature_std', 'temp_range',
    'hdd', 'cdd', 'is_very_hot', 'is_very_cold',
    # Other weather
    'humidity_max', 'humidity_min', 'humidity_mean',
    'precipitation_sum', 'precipitation_max',
    'wind_speed_max', 'wind_speed_mean',
    # Temporal features
    'month', 'day_of_week', 'day_of_month', 'week_of_year',
    'is_weekend', 'is_holiday',
    'month_sin', 'month_cos', 'dow_sin', 'dow_cos'
]
FP_COST = 1  # False Positive: predict peak when not peak
FN_COST = 4  # False Negative: miss a peak day

# Get US holidays
us_holidays = holidays.US(years=range(2016, 2026))

df['date'] = df['datetime'].dt.date

# Aggregate to daily level - WEATHER ONLY (no load features except for labeling)
daily_agg = df.groupby(['date', 'region']).agg({
    'temperature': ['max', 'min', 'mean', 'std'],
    'humidity': ['max', 'min', 'mean'],
    'precipitation': ['sum', 'max'],
    'wind_speed': ['max', 'mean']
}).reset_index()

# Flatten column names
daily_agg.columns = ['_'.join(col).strip('_') if col[1] else col[0] for col in daily_agg.columns.values]

# Convert date to datetime
daily_agg['date'] = pd.to_datetime(daily_agg['date'])

# Add temporal features
daily_agg['year'] = daily_agg['date'].dt.year
daily_agg['month'] = daily_agg['date'].dt.month
daily_agg['day_of_week'] = daily_agg['date'].dt.dayofweek
daily_agg['day_of_month'] = daily_agg['date'].dt.day
daily_agg['week_of_year'] = daily_agg['date'].dt.isocalendar().week
daily_agg['is_weekend'] = (daily_agg['day_of_week'] >= 5).astype(int)
daily_agg['is_holiday'] = daily_agg['date'].apply(lambda x: x in us_holidays).astype(int)

# Temperature features
daily_agg['temp_range'] = daily_agg['temperature_max'] - daily_agg['temperature_min']

# Heating/Cooling Degree Days (base 65°F)
daily_agg['hdd'] = np.maximum(0, 65 - daily_agg['temperature_mean'])
daily_agg['cdd'] = np.maximum(0, daily_agg['temperature_mean'] - 65)

# Extreme temperature indicators
daily_agg['is_very_hot'] = (daily_agg['temperature_max'] > 85).astype(int)
daily_agg['is_very_cold'] = (daily_agg['temperature_min'] < 32).astype(int)

# Cyclical encoding for temporal features
daily_agg['month_sin'] = np.sin(2 * np.pi * daily_agg['month'] / 12)
daily_agg['month_cos'] = np.cos(2 * np.pi * daily_agg['month'] / 12)
daily_agg['dow_sin'] = np.sin(2 * np.pi * daily_agg['day_of_week'] / 7)
daily_agg['dow_cos'] = np.cos(2 * np.pi * daily_agg['day_of_week'] / 7)

print(f"Daily aggregated data: {len(daily_agg):,} rows")
print(f"Date range: {daily_agg['date'].min()} to {daily_agg['date'].max()}")
print(f"\nFeatures created (NO LOAD FEATURES - weather and date only)")

Daily aggregated data: 319 rows
Date range: 2025-11-20 00:00:00 to 2025-11-30 00:00:00

Features created (NO LOAD FEATURES - weather and date only)


In [25]:
test_df = daily_agg[
        (daily_agg['date'] >= test_start) & 
        (daily_agg['date'] <= test_end)
    ].copy()

In [26]:
# Load the saved peak days models
regions = ['AECO', 'AEPAPT', 'AEPIMP', 'AEPKPT', 'AEPOPT', 'AP', 'BC', 'CE', 'DAY', 'DEOK',
           'DOM', 'DPLCO', 'DUQ', 'EASTON', 'EKPC', 'JC', 'ME', 'OE', 'OVEC', 'PAPWR',
           'PE', 'PEPCO', 'PLCO', 'PN', 'PS', 'RECO', 'SMECO', 'UGI', 'VMEU']

models_dir = os.path.join(OUTPUT_DIR, 'trained_models')
model_path = os.path.join(models_dir, 'peak_days_models.pkl')

print("Loading saved peak days prediction models...")
with open(model_path, 'rb') as f:
    peak_days_best_models = pickle.load(f)
print(f"Loaded {len(peak_days_best_models)} regional models")

# Dictionary to store peak days predictions
peak_days_predictions = {}

for region in tqdm(regions, desc="Predicting peak days"):
    # Get model info for this region
    model_info = peak_days_best_models[region]
    model = model_info['model']
    threshold = model_info['threshold']
    
    # Prepare prediction data for this region
    pred_data = test_df[test_df['region'] == region].copy()
    
    if len(pred_data) == 0:
        print(f"Warning: No data for {region} in prediction period")
        continue
    
    # Get features (same columns used in training)
    X_pred = pred_data[feature_cols]
    
    # Get prediction probabilities
    y_pred_proba = model.predict_proba(X_pred)[:, 1]
    
    # Apply threshold to get binary predictions
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    # Add predictions to dataframe
    pred_data['peak_day_pred'] = y_pred
    
    # Store predictions by date
    region_predictions = {}
    for idx, row in pred_data.iterrows():
        region_predictions[row['date']] = row['peak_day_pred']
    
    peak_days_predictions[region] = region_predictions

# Convert to 2D DataFrame (regions × dates)
print("\nCreating 2D DataFrame (regions × dates)...")

# Create DataFrame with regions as rows and dates as columns
peak_days_df = pd.DataFrame(peak_days_predictions).T
peak_days_df = peak_days_df.sort_index()  # Sort by region name

# Ensure columns are sorted by date
peak_days_df = peak_days_df[sorted(peak_days_df.columns)]

# Convert DatetimeIndex to datetime.date objects (match peak_hour_df format)
peak_days_df.columns = peak_days_df.columns.date 

print(f"\nPeak Days Predictions DataFrame shape: {peak_days_df.shape}")
print(f"  Rows (regions): {peak_days_df.shape[0]}")
print(f"  Columns (dates): {peak_days_df.shape[1]}")

# Display the predictions
print("\nPeak Days Predictions (0=non-peak day, 1=peak day):")
print(peak_days_df)

Loading saved peak days prediction models...
Loaded 29 regional models


Predicting peak days: 100%|███████████████████| 29/29 [00:00<00:00, 1013.56it/s]


Creating 2D DataFrame (regions × dates)...

Peak Days Predictions DataFrame shape: (29, 10)
  Rows (regions): 29
  Columns (dates): 10

Peak Days Predictions (0=non-peak day, 1=peak day):
        2025-11-20  2025-11-21  2025-11-22  2025-11-23  2025-11-24  \
AECO             1           0           0           0           1   
AEPAPT           0           0           0           0           1   
AEPIMP           1           1           0           0           1   
AEPKPT           0           0           0           0           0   
AEPOPT           1           0           0           0           1   
AP               1           0           0           0           1   
BC               0           0           0           0           1   
CE               1           0           0           0           1   
DAY              1           1           0           0           1   
DEOK             0           0           0           0           1   
DOM              1           0           

In [27]:
# Get tomorrow's date
from datetime import datetime, timedelta
tomorrow = datetime.now() + timedelta(days=1)
tomorrow_date = tomorrow.date()
tomorrow_str = tomorrow.strftime('%Y-%m-%d')

# Read from existing prediction files
predictions_dir = os.path.join(OUTPUT_DIR, 'predictions')

# Build output in required format
output_values = [f'"{tomorrow_str}"']

# 1. HOURLY LOAD PREDICTIONS (L1_00 to L29_23) from predictions_df
for region in regions:
    region_data = predictions_df[(predictions_df['date'] == tomorrow_date) & 
                                  (predictions_df['region'] == region)]
    
    if len(region_data) > 0:
        hour_cols = [f'hour_{i:02d}' for i in range(24)]
        loads = region_data[hour_cols].values[0]
        # loads = [int(round(load)) for load in loads]
        output_values.extend([str(load) for load in loads])
    else:
        output_values.extend(['0'] * 24)

# 2. PEAK HOUR PREDICTIONS (PH_1 to PH_29) from peak_hour_df
for region in regions:
    if tomorrow_date in peak_hour_df.columns and region in peak_hour_df.index:
        peak_hour = int(peak_hour_df.loc[region, tomorrow_date])
        output_values.append(str(peak_hour))
    else:
        output_values.append('0')

# 3. PEAK DAY PREDICTIONS (PD_1 to PD_29) from peak_days_df
for region in regions:
    if tomorrow_date in peak_days_df.columns and region in peak_days_df.index:
        peak_day = int(peak_days_df.loc[region, tomorrow_date])
        output_values.append(str(peak_day))
    else:
        output_values.append('0')
# Create the output line
output_line = ','.join(output_values)

In [28]:
print(output_line)

"2025-11-22",820.0,800.0,779.0,765.0,772.0,787.0,835.0,852.0,863.0,913.0,887.0,890.0,889.0,903.0,929.0,948.0,1021.0,1112.0,1128.0,1103.0,1081.0,1068.0,995.0,941.0,3448.0,3385.0,3358.0,3340.0,3361.0,3431.0,3590.0,3743.0,3873.0,3942.0,3983.0,3947.0,3997.0,4029.0,4033.0,4102.0,4134.0,4149.0,4251.0,4310.0,4247.0,4235.0,4130.0,4031.0,2565.0,2519.0,2470.0,2447.0,2465.0,2492.0,2647.0,2717.0,2730.0,2791.0,2786.0,2823.0,2813.0,2792.0,2792.0,2787.0,2812.0,2831.0,2879.0,2899.0,2894.0,2856.0,2754.0,2642.0,529.0,528.0,527.0,533.0,543.0,545.0,569.0,589.0,597.0,609.0,614.0,616.0,625.0,616.0,616.0,618.0,622.0,623.0,656.0,664.0,679.0,689.0,676.0,659.0,5181.0,5032.0,4956.0,4947.0,4943.0,5071.0,5227.0,5532.0,5789.0,5993.0,5931.0,5903.0,5896.0,5833.0,5834.0,5841.0,5883.0,6165.0,6295.0,6319.0,6246.0,6259.0,6018.0,5799.0,4354.0,4244.0,4194.0,4198.0,4199.0,4297.0,4481.0,4767.0,4901.0,4918.0,5036.0,4968.0,5009.0,5049.0,5027.0,5155.0,5227.0,5352.0,5384.0,5424.0,5465.0,5393.0,5135.0,4893.0,2694.0,2625.0,2590.0,